In [ ]:
import cv2
import dlib
import numpy as np
import tensorflow as tf

# Carregar detector de faces pré-treinado
detector = dlib.get_frontal_face_detector()

# Carregar modelo de classificação facial pré-treinado
model = tf.keras.models.load_model('modelo_classificacao_facial.h5')

# Função para detectar e reconhecer faces
def detect_and_recognize_faces(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        face_image = gray[y:y+h, x:x+w]
        face_image = cv2.resize(face_image, (160, 160))  # Redimensionar para o tamanho esperado pelo modelo

        # Normalizar imagem
        face_image = face_image / 255.0

        # Adicionar dimensão extra para corresponder ao formato de entrada do modelo
        face_image = np.expand_dims(face_image, axis=0)
        face_image = np.expand_dims(face_image, axis=-1)

        # Realizar a predição usando o modelo de classificação facial
        prediction = model.predict(face_image)

        # Exemplo de saída: ['pessoa_1', 'pessoa_2', 'pessoa_3', ...]
        predicted_label = np.argmax(prediction)

        # Desenhar caixa delimitadora e nome da classe reconhecida na imagem
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(image, f'Pessoa {predicted_label}', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    return image

# Carregar imagem de teste
image_path = 'test_image.jpg'
image = cv2.imread(image_path)

# Detectar e reconhecer faces na imagem de teste
result_image = detect_and_recognize_faces(image)

# Exibir resultado
cv2.imshow('Resultado', result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
